In [ ]:
%pip install google-generativeai
import os
import dotenv
import google.generativeai as genai

dotenv.load_dotenv()
    
def generate_output(system_prompt, user_prompt):
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        system_instruction=system_prompt
    )

    response = model.generate_content(
        user_prompt,
        generation_config={
            "temperature": 0
        }
    )

    return response.text

In [13]:
import re
import time
from tqdm import tqdm
def replace_multi_table(text, replace_text):
    
    table_pattern = r'(\|.*\|(?:\n\|.*\|)+)'
    table = re.findall(table_pattern, text)
    text = text.replace(table[0], replace_text)
        
    return text

In [ ]:
import pandas as pd
import time
from utils.multihiertt_table_extraction import MultiHierttTableExtractor

instruction = "Retrieve all facts in the following table as natural language descriptions, without introductory phrases."
df = pd.read_csv("./financerag/dataset_csv/MultiHiertt_corpus.csv")

call_count = 0

for idx in range(len(df)):
    text = df.at[idx, "text"]

    if not isinstance(text, str):
        continue

    if MultiHierttTableExtractor.has_table(text):
        result = MultiHierttTableExtractor.extract_table(text)
        if result:
            _, table_content = result
            parsed = MultiHierttTableExtractor.parse_table_to_text(table_content)
            replaced = generate_output(instruction, parsed)
            new_text = replace_multi_table(text, replaced)

            # update only this row
            df.at[idx, "text"] = new_text

            call_count += 1
            if call_count % 1000 == 0:
                time.sleep(60)

# write final file
df.to_csv("./financerag/dataset_csv/MultiHiertt_corpus_cleaned.csv", index=False)


[['(in millions, except per share data)', '2003', '2002', '2001'], ['Basic earnings per share', '', '', ''], ['Net income', '$6,719', '$1,663', '$1,694'], ['Less: Preferred stock dividends', '51', '51', '66'], ['Net income applicable to common stock', '$6,668', '$1,612', '$1,628'], ['Weighted-average basic shares outstanding', '2,008.6', '1,984.3', '1,972.4'], ['Net income per share', '$3.32', '$0.81', '$0.83'], ['Diluted earnings per share', '', '', ''], ['Net income applicable to common stock', '$6,668', '$1,612', '$1,628'], ['Weighted-average basic shares outstanding', '2,008.6', '1,984.3', '1,972.4'], ['Add: Broad-based options', '4.1', '2.8', '6.6'], ['Key employee options', '42.4', '22.0', '44.6'], ['Weighted-average diluted shares outstanding', '2,055.1', '2,009.1', '2,023.6'], ['Net income per share(a)', '$3.24', '$0.80', '$0.80']]
[['', 'Year Ended October 31,', '$ Change', '% Change', '$ Change', '% Change'], ['', '2013', '2012', '2011', '2012 to 2013', '2011 to 2012'], ['', 

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 250, model: gemini-2.5-flash
Please retry in 280.785608ms. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250
}
, retry_delay {
}
]